In [3]:
import numpy as np
from PIL import Image

def Interferogram_Plot(D, C, B, G, F, J, E, I, pixels, _):
    """
    Generate an interferogram plot with the given parameters.
    
    Parameters same as MATLAB version:
    D: Defocus
    C: Tilt(x)
    B: Tilt(y)
    G: Spherical
    F: Coma(y)
    J: Coma(x)
    E: Astig(y)
    I: Astig(x)
    pixels: Resolution (square)
    _: Unused parameter (kept for compatibility)
    """
    
    respix = pixels
    
    WFE = np.zeros((respix, respix))
    
    x = np.linspace(-(respix/2 - 0.5)/(respix/2), (respix/2 - 0.5)/(respix/2), respix)
    y = np.linspace(-(respix/2 - 0.5)/(respix/2), (respix/2 - 0.5)/(respix/2), respix)
    X, Y = np.meshgrid(x, y)
    
    OPD = (B * X + C * Y + 
           D * (X**2 + Y**2) + 
           E * (X**2 + 3*Y**2) + 
           F * Y * (X**2 + Y**2) + 
           G * (X**2 + Y**2)**2 + 
           J * X * (X**2 + Y**2) + 
           I * (3*X**2 + Y**2))
    
    WFE = OPD
    
    phase = 1 - np.abs(0.5 - (WFE - np.floor(WFE)))/0.5
    
    grey_plot = Image.fromarray((phase * 255).astype(np.uint8))
    
    return grey_plot

In [4]:
import os
import numpy as np

cap = 5
num_images = 4000
res = 224

if not os.path.exists('training'):
    os.makedirs('training')

np.random.seed() 

for img in range(num_images):
    params = np.round(np.random.uniform(-cap, cap, 8), 6)

    image = Interferogram_Plot(params[0], params[1], params[2], params[3],
                             params[4], params[5], params[6], params[7],
                             res, None)
    
    base_filename = f'img_D{params[0]:.6f}_C{params[1]:.6f}_B{params[2]:.6f}_G{params[3]:.6f}_' \
                   f'F{params[4]:.6f}_J{params[5]:.6f}_E{params[6]:.6f}_I{params[7]:.6f}'
    
    base_filename = base_filename.replace('-', 'n').replace('.', 'p')
    
    write_file = os.path.join('training', f'{base_filename}.jpg')
    image.save(write_file)

In [1]:
import sys
print(sys.executable)
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import numpy as np
import glob

class InterferogramDataset(Dataset):
    def __init__(self, file_paths):
        self.file_paths = file_paths
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Grayscale(1)
        ])
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path)
        image = self.transform(image)
        
        name = os.path.splitext(os.path.basename(img_path))[0]
        name = name.replace('n', '-').replace('p', '.')
        parts = name.split('_')
        params = np.zeros(8)
        for i, part in enumerate(parts[1:9]):
            params[i] = float(part[1:])
            
        return image, torch.FloatTensor(params)

class InterferogramNet(nn.Module):
    def __init__(self):
        super(InterferogramNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 112 * 112, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 8)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

def preprocess_image(filename):
    image = Image.open(filename).convert('L')
    image = np.array(image, dtype=np.float32) / 255.0
    return image

def output_function(epoch, iteration, train_loss, train_rmse, val_loss=None, val_rmse=None, 
                   time_since_start=0, learning_rate=0):
    if iteration % 100 == 0:
        print(f'\n=== Training Progress at Iteration {iteration} ===')
        print(f'Epoch: {epoch}')
        print(f'Training Loss: {train_loss:.6f}')
        print(f'Training RMSE: {train_rmse:.6f}')
        if val_loss is not None:
            print(f'Validation Loss: {val_loss:.6f}')
            print(f'Validation RMSE: {val_rmse:.6f}')
        print(f'Time Since Start: {time_since_start/60:.2f} minutes')
        print(f'Current Learning Rate: {learning_rate:.6f}')
        print('===================================\n')

def main
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    file_paths = glob.glob(os.path.join('training', '*.jpg'))
    num_files = len(file_paths)
    print(f'Number of files: {num_files}')
    
    full_dataset = InterferogramDataset(file_paths)
    
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=14, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=14, shuffle=False, num_workers=4)
    
    model = InterferogramNet().to(device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    num_epochs = 5
    iteration = 0
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (images, targets) in enumerate(train_loader):
            images, targets = images.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            rmse = torch.sqrt(loss)
            iteration += 1
            
            if iteration % 5 == 0:
                model.eval()
                val_loss = 0
                val_rmse = 0
                with torch.no_grad():
                    for val_images, val_targets in val_loader:
                        val_images, val_targets = val_images.to(device), val_targets.to(device)
                        val_outputs = model(val_images)
                        val_batch_loss = criterion(val_outputs, val_targets)
                        val_loss += val_batch_loss.item()
                        val_rmse += torch.sqrt(val_batch_loss).item()
                
                val_loss /= len(val_loader)
                val_rmse /= len(val_loader)
                
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= 1:
                        print("Early stopping triggered")
                        break
                
                output_function(epoch, iteration, loss.item(), rmse.item(),
                              val_loss, val_rmse, iteration * 0.1, optimizer.param_groups[0]['lr'])
                
                model.train()
    
    save_dir = 'models'
    os.makedirs(save_dir, exist_ok=True)
    model_path = os.path.join(save_dir, 'trained_network.pth')
    torch.save(model.state_dict(), model_path)
    print(f'Model saved as: {model_path}')
    
    print('\n=== Testing Some Predictions ===')
    model.eval()
    with torch.no_grad():
        test_images, actual_params = next(iter(val_loader))
        test_images, actual_params = test_images.to(device), actual_params.to(device)
        predictions = model(test_images)
        
        for i in range(min(5, predictions.size(0))):
            print(f'\nSample {i+1}:')
            print(f'Predicted: D={predictions[i,0]:.4f}, C={predictions[i,1]:.4f}, '
                  f'B={predictions[i,2]:.4f}, G={predictions[i,3]:.4f}, '
                  f'F={predictions[i,4]:.4f}, J={predictions[i,5]:.4f}, '
                  f'E={predictions[i,6]:.4f}, I={predictions[i,7]:.4f}')
            print(f'Actual:    D={actual_params[i,0]:.4f}, C={actual_params[i,1]:.4f}, '
                  f'B={actual_params[i,2]:.4f}, G={actual_params[i,3]:.4f}, '
                  f'F={actual_params[i,4]:.4f}, J={actual_params[i,5]:.4f}, '
                  f'E={actual_params[i,6]:.4f}, I={actual_params[i,7]:.4f}')
            
            errors = torch.abs(predictions[i] - actual_params[i])
            print(f'Abs Error: D={errors[0]:.4f}, C={errors[1]:.4f}, '
                  f'B={errors[2]:.4f}, G={errors[3]:.4f}, '
                  f'F={errors[4]:.4f}, J={errors[5]:.4f}, '
                  f'E={errors[6]:.4f}, I={errors[7]:.4f}')

if __name__ == '__main__':
    main()

/work/csse463/202520/06/myenv/bin/python
Number of files: 40
Model saved as: models/trained_network.pth

=== Testing Some Predictions ===

Sample 1:
Predicted: D=0.3538, C=0.1651, B=-0.0904, G=0.6991, F=-0.2561, J=-0.6439, E=0.9614, I=-0.1587
Actual:    D=1.5358, C=3.5613, B=-4.7948, G=-1.7157, F=-4.9548, J=-3.5965, E=1.0944, I=-3.1230
Abs Error: D=1.1820, C=3.3962, B=4.7044, G=2.4149, F=4.6987, J=2.9527, E=0.1330, I=2.9642

Sample 2:
Predicted: D=0.2499, C=0.1048, B=0.0824, G=0.5808, F=-0.2120, J=-0.4875, E=0.6757, I=-0.1483
Actual:    D=3.7628, C=-4.2970, B=0.8222, G=-4.5054, F=-0.3217, J=1.6811, E=-3.3811, I=-1.9742
Abs Error: D=3.5129, C=4.4018, B=0.7398, G=5.0862, F=0.1097, J=2.1686, E=4.0568, I=1.8259

Sample 3:
Predicted: D=0.1733, C=0.0604, B=0.0781, G=0.4662, F=-0.1367, J=-0.3778, E=0.5268, I=-0.1460
Actual:    D=4.2930, C=-4.4883, B=-3.4058, G=4.5360, F=1.2139, J=-0.5252, E=4.0946, I=-1.0538
Abs Error: D=4.1197, C=4.5487, B=3.4839, G=4.0698, F=1.3506, J=0.1474, E=3.5677, I=0.